# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import googlemaps

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# File to Load
file_to_load = "../output_data/cities.csv"
# Read Purchasing File and store into Pandas data frame
weather_df = pd.read_csv(file_to_load)
weather_df


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [15]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)
 # Store 'Lat' and 'Lng' into  locations
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)
#plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
fig=gmaps.figure()

In [16]:
#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_df.loc[(weather_df["Wind Speed"] <= 25) & (weather_df["Cloudiness"] == 0) & \
                                 (weather_df["Max Temp"] >= 80)].dropna()

narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
26,26,colonelganj,0,IN,1558378759,14,27.13,81.70,94.20,5.08
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
142,142,sarai mir,0,IN,1558378781,35,26.02,82.92,86.00,3.36
244,244,pasni,0,PK,1558378823,52,25.26,63.48,86.82,9.17
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17
404,404,manama,0,AE,1558378872,52,25.33,56.02,91.40,14.99
412,412,diu,0,IN,1558378873,75,20.71,70.98,83.04,12.82
416,416,kirkuk,0,IQ,1558378874,16,35.47,44.40,86.82,7.36
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
527,527,tomatlan,0,MX,1558378894,59,19.93,-105.25,84.66,7.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df


,City,Country,Lat,Lng,Hotel Name
26,colonelganj,IN,27.13,81.70,
125,arraial do cabo,BR,-22.97,-42.02,
142,sarai mir,IN,26.02,82.92,
244,pasni,PK,25.26,63.48,
316,gunjur,GM,13.18,-16.76,
404,manama,AE,25.33,56.02,
412,diu,IN,20.71,70.98,
416,kirkuk,IQ,35.47,44.40,
518,bonito,BR,-21.13,-56.49,
527,tomatlan,MX,19.93,-105.25,


In [8]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "name",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
        print("------------")
    

# Print end of search once searching is completed
#print("-------End of Search-------")


Retrieving Results for Index 26: colonelganj.
Closest hotel in colonelganj is Anand Hotel.
Retrieving Results for Index 125: arraial do cabo.
Closest hotel in arraial do cabo is Orlanova Hotel.
Retrieving Results for Index 142: sarai mir.
Closest hotel in sarai mir is Bhola T Center.
Retrieving Results for Index 244: pasni.
Closest hotel in pasni is Juddi Hotel.
Retrieving Results for Index 316: gunjur.
Closest hotel in gunjur is Dalaba Eco-Lodge.
Retrieving Results for Index 404: manama.
Closest hotel in manama is اوتيل City Stay.
Retrieving Results for Index 412: diu.
Closest hotel in diu is Azzaro Resorts & Ganga Spa.
Retrieving Results for Index 416: kirkuk.
Closest hotel in kirkuk is Morgan Hotel.
Retrieving Results for Index 518: bonito.
Closest hotel in bonito is Pousada Arte da Natureza.
Retrieving Results for Index 527: tomatlan.
Closest hotel in tomatlan is Hotel Hacienda Vieja.
Retrieving Results for Index 543: yabrud.
Missing field/result... skipping.
------------


In [10]:
#Hotel dataframe
hotel_df

,City,Country,Lat,Lng,Hotel Name
26,colonelganj,IN,27.13,81.70,Anand Hotel
125,arraial do cabo,BR,-22.97,-42.02,Orlanova Hotel
142,sarai mir,IN,26.02,82.92,Bhola T Center
244,pasni,PK,25.26,63.48,Juddi Hotel
316,gunjur,GM,13.18,-16.76,Dalaba Eco-Lodge
404,manama,AE,25.33,56.02,اوتيل City Stay
412,diu,IN,20.71,70.98,Azzaro Resorts & Ganga Spa
416,kirkuk,IQ,35.47,44.40,Morgan Hotel
518,bonito,BR,-21.13,-56.49,Pousada Arte da Natureza
527,tomatlan,MX,19.93,-105.25,Hotel Hacienda Vieja


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))